# Tarea 1

## Luis Felipe Cubero Vargas

El siguiente documento corresponde a la tarea 1 del curso de Bigdata 

Lo primero a realizar es la carga de paquetes necesarios

In [1]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('data_processing').getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.types import *

In [2]:
import findspark
findspark.init('/opt/spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType

Una vez cargados los paquete necesarios se procede con la carga del dataset que se pretende analizar.

En este caso consiste en una dataset sobre el prestamos de bicicletas en londres, tomando en cuando la season, la temperatura y su sensación termica, así como también la humedad y la velocidad del viento, si es feriado, si es fin de semana.

In [3]:
df = spark.read.csv('london_merged.csv',header = True, inferSchema = True)

In [5]:
df.schema

StructType(List(StructField(timestamp,TimestampType,true),StructField(cnt,IntegerType,true),StructField(t1,DoubleType,true),StructField(t2,DoubleType,true),StructField(hum,DoubleType,true),StructField(wind_speed,DoubleType,true),StructField(weather_code,DoubleType,true),StructField(is_holiday,DoubleType,true),StructField(is_weekend,DoubleType,true),StructField(season,DoubleType,true)))

In [4]:
df.show(5)

+-------------------+---+---+---+-----+----------+------------+----------+----------+------+
|          timestamp|cnt| t1| t2|  hum|wind_speed|weather_code|is_holiday|is_weekend|season|
+-------------------+---+---+---+-----+----------+------------+----------+----------+------+
|2015-01-04 00:00:00|182|3.0|2.0| 93.0|       6.0|         3.0|       0.0|       1.0|   3.0|
|2015-01-04 01:00:00|138|3.0|2.5| 93.0|       5.0|         1.0|       0.0|       1.0|   3.0|
|2015-01-04 02:00:00|134|2.5|2.5| 96.5|       0.0|         1.0|       0.0|       1.0|   3.0|
|2015-01-04 03:00:00| 72|2.0|2.0|100.0|       0.0|         1.0|       0.0|       1.0|   3.0|
|2015-01-04 04:00:00| 47|2.0|0.0| 93.0|       6.5|         1.0|       0.0|       1.0|   3.0|
+-------------------+---+---+---+-----+----------+------------+----------+----------+------+
only showing top 5 rows



In [19]:
df.columns

['timestamp',
 'cnt',
 't1',
 't2',
 'hum',
 'wind_speed',
 'weather_code',
 'is_holiday',
 'is_weekend',
 'season',
 'temperature_feels']

Con lo anterior se nos permite visualizar el tipo de datos que tenemos una vez cargado la base de datos.
Ahora realizaremos una  busqueda para ver si existen valores nulos dentro del dataset

In [6]:
df.where(col("timestamp").isNull()).count()
for c in df.columns: print(c,df.where(col(c).isNull()).count())

timestamp 0
cnt 0
t1 0
t2 0
hum 0
wind_speed 0
weather_code 0
is_holiday 0
is_weekend 0
season 0


De lo anterior podemos observar que no existen valores nulos dentro del dataset con lo cual podemos continuar con el análisis

In [7]:
df.summary().show()

+-------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+-------------------+------------------+
|summary|               cnt|                t1|               t2|               hum|        wind_speed|      weather_code|         is_holiday|         is_weekend|            season|
+-------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+-------------------+------------------+
|  count|             17414|             17414|            17414|             17414|             17414|             17414|              17414|              17414|             17414|
|   mean|1143.1016423567244|12.468090808162016|11.52083572604418| 72.32495405995176|15.913063244133076| 2.722751808889399|0.02205122315378431|0.28540254967267714|1.4920753416791088|
| stddev|1085.1080679362196|  5.57181756292053|6.615144673127889|14.313186095310893|7.8945

In [8]:
df.filter(df['is_weekend']==1.0).agg(F.mean('cnt')).show()

+---------------+
|       avg(cnt)|
+---------------+
|977.41569416499|
+---------------+



In [9]:
df.filter(df['is_weekend']==0.0).agg(F.mean('cnt')).show()

+-----------------+
|         avg(cnt)|
+-----------------+
|1209.274831243973|
+-----------------+



De lo anterior podemos ver que la gente tiende a utilizar más las bicicletas durante la semana que durante los fines de semana, lo cual puede ser un indicador importante para el comportamiento del prestamos de bicicletas, ahora veremos si se cumple lo mismo para los diás feriados

In [10]:
df.filter(df['is_holiday']==1.0).agg(F.mean('cnt')).show()

+-----------------+
|         avg(cnt)|
+-----------------+
|769.5260416666666|
+-----------------+



In [11]:
df.filter(df['is_holiday']==0.0).agg(F.mean('cnt')).show()

+------------------+
|          avg(cnt)|
+------------------+
|1151.5251908396947|
+------------------+



Nuevamente podemos observar que durante los días feriados el patrón se mantiene. 

In [12]:
df.select("weather_code").distinct().show()


+------------+
|weather_code|
+------------+
|         7.0|
|         1.0|
|         4.0|
|         3.0|
|         2.0|
|        10.0|
|        26.0|
+------------+



In [13]:
df = df.withColumn(
    'temperature_feels',
    F.when(F.col('t2').between(-10.0,0.0 ), '-10 to 0')\
    .when( F.col('t2').between(0.0,10.0), '0 to 10')\
    .when(F.col('t2').between(10.0,20.0), '10 to 20')\
    .when(F.col('t2').between(20,34), '20 to 34')\
    .otherwise(0)
)

Con el condigo anterior agrupamos la sensación termica en 4 grupos lo cual nos permite ver el comportamiento del alquiler de bicicletas basado en la sensanción terminca del día

In [14]:
df.groupby('temperature_feels').agg(F.mean('cnt')).show()

+-----------------+------------------+
|temperature_feels|          avg(cnt)|
+-----------------+------------------+
|         20 to 34| 2342.528354080221|
|         10 to 20|1205.9541612483745|
|         -10 to 0| 645.3968042609854|
|          0 to 10|  819.074636834196|
+-----------------+------------------+



De lo anterior podemos ver que el alquiler de bicicletas tiene un crecimiento lineal a medida que aumenta la temperatura

In [15]:
df = df.withColumn("weather_code", \
              F.when(df["weather_code"] == 1.0, 'Clear')\
                .when(df["weather_code"]==2.0,'Scattered clouds / few clouds')\
                .when(df["weather_code"]==3.0,'Broken clouds') \
                .when(df["weather_code"]==4.0,'Cloudy')\
                .when(df["weather_code"]==7.0,'Rain/ light Rain shower/ Light rain') \
                .when(df["weather_code"]==10.0,'Rain with thunderstorm') \
                .when(df["weather_code"]==26.0,'Snowfall') \
                   .otherwise(df["weather_code"]))

In [16]:
df.groupby('temperature_feels','weather_code').agg(F.mean('cnt')).show()

+-----------------+--------------------+------------------+
|temperature_feels|        weather_code|          avg(cnt)|
+-----------------+--------------------+------------------+
|          0 to 10|Rain with thunder...| 762.1666666666666|
|         -10 to 0|Scattered clouds ...|        977.890625|
|          0 to 10|            Snowfall|           138.125|
|          0 to 10|              Cloudy| 591.1295487627366|
|          0 to 10|               Clear| 745.7786370227163|
|         -10 to 0|               Clear| 530.7789473684211|
|         -10 to 0|       Broken clouds| 874.6974789915967|
|         10 to 20|Rain/ light Rain ...| 787.0081566068516|
|          0 to 10|Scattered clouds ...|1087.7340764331211|
|         20 to 34|              Cloudy|           1677.25|
|         10 to 20|               Clear|1200.3531799729365|
|         10 to 20|            Snowfall|              38.0|
|         10 to 20|       Broken clouds|1233.5009372071227|
|         10 to 20|Rain with thunder...|

De analisis anterior en donde consideramos el clima y la sensación termica, podemos ver como se comporta el alquiler en donde el factor de que clima cambia la decision de alquilar o no, podemos ver que por ejemplo en casos donde la sensación termica esta 10 a 20 pero esta necando la gente decide no alquilar, lo cual nos indica una correlación entre las variables y por ende como factores predictores del alquiler o prestamo de bicicletas.

Ahora vamos a generar una matriz de correlaciones de pearson para ver como interactuan las diferentes variables

In [26]:
from pyspark.mllib.stat import Statistics
import pandas as pd

# df = sqlCtx.read.format('com.databricks.spark.csv').option('header', 'true').option('inferschema', 'true').load('corr_test.csv')
df = df.select(['cnt','t1','t2','hum', 'wind_speed', 'is_holiday', 'is_weekend', 'season'])
col_names = df.columns
features = df.rdd.map(lambda row: row[0:])
corr_mat=Statistics.corr(features, method="pearson")
corr_df = pd.DataFrame(corr_mat,columns=col_names,index=col_names)

In [27]:
corr_df

,cnt,t1,t2,hum,wind_speed,is_holiday,is_weekend,season
cnt,1.000000,0.388798,0.369035,-0.462901,0.116295,-0.051698,-0.096499,-0.116180
t1,0.388798,1.000000,0.988344,-0.447781,0.145471,-0.042233,-0.005342,-0.285851
t2,0.369035,0.988344,1.000000,-0.403495,0.088409,-0.040051,-0.008510,-0.285900
hum,-0.462901,-0.447781,-0.403495,1.000000,-0.287789,0.032068,0.028098,0.290381
wind_speed,0.116295,0.145471,0.088409,-0.287789,1.000000,-0.002606,0.011479,0.010305
is_holiday,-0.051698,-0.042233,-0.040051,0.032068,-0.002606,1.000000,-0.094898,-0.032488
is_weekend,-0.096499,-0.005342,-0.008510,0.028098,0.011479,-0.094898,1.000000,0.001067
season,-0.116180,-0.285851,-0.285900,0.290381,0.010305,-0.032488,0.001067,1.000000


De lo anterior como esperabamos la temperatura tiene una correlación positiva con el prestamos de bicicletas, al igual que la velocidad del viento, de igual forma si es feriado o fin de semana tienen un relación negativa pero se puede observar que es una relaxión pequeña

In [46]:
df2 = df.select('cnt').rdd.map(lambda x: (x,1)).to_DF()


AttributeError: 'PipelinedRDD' object has no attribute 'to_DF'